# VTR to VTK conversion for Blender
---
This is a small utility written to take .vtr files (for example those exported from lumerical) and turn it into vtr data that can be used to perform a time-dependent visualization in paraview (and then exported to blender)


In [ ]:
import vtk
from vtk.util import numpy_support
import numpy as np
import os
import xml.etree.ElementTree as ET


In [ ]:

# Step 1: Read the VTR file and extract the 'Pz' array
filename = 'P_downsampled.vtr'  # Replace with your actual file path
reader = vtk.vtkXMLRectilinearGridReader()
reader.SetFileName(filename)
reader.Update()
data = reader.GetOutput()


In [ ]:

# Extract the 'Pz' array
pz_vtk_array = data.GetPointData().GetArray('Pz')
if not pz_vtk_array:
    print("Error: 'Pz' array not found in Point Data.")
    exit(1)

# Convert to NumPy array
pz_numpy = numpy_support.vtk_to_numpy(pz_vtk_array)

# Determine the number of points and time steps
num_points, num_time_steps = pz_numpy.shape
print(f"Pz array shape: {pz_numpy.shape} (Points: {num_points}, Time Steps: {num_time_steps})")

# Step 2: Create a time array from 0 to 100 fs
time_values = np.linspace(0, 100, num_time_steps)  # Time in femtoseconds

# Step 3: Loop over each time step to create individual VTR files
output_dir = 'output_vtr_files'
os.makedirs(output_dir, exist_ok=True)

# Get coordinate information
x_coords = data.GetXCoordinates()
y_coords = data.GetYCoordinates()
z_coords = data.GetZCoordinates()
dimensions = data.GetDimensions()


In [ ]:

for i in range(num_time_steps):
    # Extract data for the current time step
    pz_time_step = pz_numpy[:, i]

    # Create a new VTK array
    pz_vtk_time_step = numpy_support.numpy_to_vtk(pz_time_step, deep=True)
    pz_vtk_time_step.SetName('Pz')

    # Create a new rectilinear grid
    grid = vtk.vtkRectilinearGrid()
    grid.SetDimensions(dimensions)
    grid.SetXCoordinates(x_coords)
    grid.SetYCoordinates(y_coords)
    grid.SetZCoordinates(z_coords)

    # Add the Pz array to the grid's point data
    grid.GetPointData().AddArray(pz_vtk_time_step)

    # Write the grid to a VTR file
    writer = vtk.vtkXMLRectilinearGridWriter()
    timestep_filename = os.path.join(output_dir, f'timestep_{i:04d}.vtr')
    writer.SetFileName(timestep_filename)
    writer.SetInputData(grid)
    writer.Write()

    print(f"Written time step {i+1}/{num_time_steps} to '{timestep_filename}'")

# Step 4: Create a PVD file to reference the time series
root = ET.Element('VTKFile', type='Collection', version='0.1', byte_order='LittleEndian')
collection = ET.SubElement(root, 'Collection')

for i, time in enumerate(time_values):
    dataset = ET.SubElement(
        collection, 'DataSet',
        timestep=str(time),
        group='',
        part='0',
        file=os.path.join('output_vtr_files', f'timestep_{i:04d}.vtr')
    )

pvd_filename = 'time_series.pvd'
tree = ET.ElementTree(root)
tree.write(pvd_filename)

print(f"PVD file '{pvd_filename}' has been created.")

# Step 5: Instructions to load the PVD file into ParaView
print("\nTo visualize the data:")
print("1. Open ParaView.")
print(f"2. Go to 'File' > 'Open' and select '{pvd_filename}'.")
print("3. Click 'Apply' in the Properties panel.")
print("4. Use the animation controls to play through the time steps.")
